In [ ]:
import os
import pickle
import numpy as np
import pandas as pd

# # Define the path to the dataset
# dataset_path = "WESAD"

# # Loop through subjects from s2 to s17, excluding s12
# for subject_id in range(2, 18):
#     if subject_id == 12:
#         continue  # Skip s12

#     # Construct the path to the subject data
#     subject_path = os.path.join(dataset_path, f"S{subject_id}")
#     subject_data_path = os.path.join(subject_path, f"S{subject_id}.pkl")

#     # Load the subject data
#     with open(subject_data_path, 'rb') as file:
#         subject_data = pickle.load(file, encoding='latin1')

#     # Extract features (modify this part based on your feature extraction logic)
#     chest_ax = subject_data['signal']['chest']['ACC'][:, 0]
#     chest_ay = subject_data['signal']['chest']['ACC'][:, 1]
#     chest_az = subject_data['signal']['chest']['ACC'][:, 2]
#     chest_ecg = subject_data['signal']['chest']['ECG'][:, 0]
#     chest_emg = subject_data['signal']['chest']['EMG'][:, 0]
#     chest_eda = subject_data['signal']['chest']['EDA'][:, 0]
#     chest_temp = subject_data['signal']['chest']['Temp'][:, 0]
#     chest_resp = subject_data['signal']['chest']['Resp'][:, 0]

#     # Assuming label information is available, replace 'label_placeholder' with actual label data
#     label = subject_data['label']

#     # Create a DataFrame
#     numpy_data = np.array([chest_ax, chest_ay, chest_az, chest_ecg, chest_emg, chest_eda, chest_temp, chest_resp, label])
#     numpy_data = numpy_data.T
#     df_subject = pd.DataFrame(data=numpy_data, columns=["chest_ax", "chest_ay", "chest_az", "chest_ecg", "chest_emg", "chest_eda", "chest_temp", "chest_resp", "label"])

#     # Output to CSV
#     output_csv_path = f'WESAD/subject_{subject_id}_data.csv'
#     df_subject.to_csv(output_csv_path, index=False)

#     print(f"Feature extraction and CSV output for subject {subject_id} completed.")

# print("Feature extraction and CSV output for all subjects completed.")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dropout, Dense
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dropout, Dense
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
from keras.optimizers import Adam
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GRU, Dropout, Dense, Flatten
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import TimeSeriesSplit
# # Load preprocessed data
# data_paths = [
# #     "/kaggle/input/csv-wesad/subject_3_data.csv",
#     "/kaggle/input/csv-wesad/subject_4_data.csv",
#     "/kaggle/input/csv-wesad/subject_5_data.csv",
#     "/kaggle/input/csv-wesad/subject_6_data.csv",
#     "/kaggle/input/csv-wesad/subject_7_data.csv",
#     "/kaggle/input/csv-wesad/subject_8_data.csv",
#     "/kaggle/input/csv-wesad/subject_9_data.csv",
#     "/kaggle/input/csv-wesad/subject_10_data.csv",
#     "/kaggle/input/csv-wesad/subject_11_data.csv",
#     "/kaggle/input/csv-wesad/subject_13_data.csv",
#     "/kaggle/input/csv-wesad/subject_14_data.csv",
#     "/kaggle/input/csv-wesad/subject_15_data.csv",
#     "/kaggle/input/csv-wesad/subject_16_data.csv",
#     "/kaggle/input/csv-wesad/subject_17_data.csv",
# ]


In [ ]:
merged_data = pd.read_csv("/kaggle/input/wesad-merged-csv/merged_data.csv")
# Assuming the last column contains the target labels
X = merged_data.iloc[:, :-1].values
y = merged_data.iloc[:, -1].values

# Reshape the input data to fit the Conv1D model
X = X.reshape(X.shape[0], X.shape[1], 1)

# Create time series split
tscv = TimeSeriesSplit(n_splits=2)

# Create the model outside the loop
model = Sequential()

# CNN layers
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', input_shape=(X.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=2))
# model.add(Flatten())

# LSTM layers with dropout
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.3))

# Fully connected layers
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(8, activation='softmax'))

# Compile the model only once
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train and evaluate the model for each split
for train_index, test_index in tscv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train the model
    history = model.fit(X_train, y_train, epochs=25, batch_size=512, validation_data=(X_test, y_test))


In [ ]:
# Plot training & validation accuracy values
plt.figure(figsize=(12, 6))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()